# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [12]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('data\\shopee_reviews.csv')

# Function to modify the label based on the rules
def modify_label(value):
    # Convert to string and strip whitespace to handle both numeric and string types
    str_value = str(value).strip()
    if str_value in ['1', '2']:
        return 'Neg'
    elif str_value == '3':
        return 'Neu'
    elif str_value in ['4', '5']:
        return 'Pos'
    else:
        return value  # Keep unchanged if it doesn't match

# Apply the modification to the 'label' column
df['label'] = df['label'].apply(modify_label)

# Reduce the number of 'Pos' records to 100,000 by random sampling
pos_df = df[df['label'] == 'Pos']
if len(pos_df) > 55000:
    pos_df = pos_df.sample(n=55000, random_state=42)  # random_state for reproducibility

# Keep all other records
other_df = df[df['label'] != 'Pos']

# Combine the reduced 'Pos' with others
modified_df = pd.concat([pos_df, other_df], ignore_index=True)

# Optionally, save the modified DataFrame to a new CSV file
modified_df.to_csv('modified_shopee_reviews.csv', index=False)

# Print a summary
print("Original shape:", df.shape)
print("Modified shape:", modified_df.shape)
print(modified_df['label'].value_counts())

C:\Users\User\AppData\Local\Temp\ipykernel_24480\1244998731.py:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data\\shopee_reviews.csv')


Original shape: (1502575, 2)
Modified shape: (151514, 2)
label
Pos      55000
Neu      49083
Neg      47430
label        1
Name: count, dtype: int64


In [16]:
import nltk

# Tải xuống resource cần thiết
nltk.download('punkt_tab')
nltk.download('punkt')        # Vẫn nên tải để đảm bảo
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [20]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# === TẢI DỮ LIỆU NLTK (chỉ cần chạy 1 lần) ===
nltk.download('punkt_tab', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)

# === ĐỌC FILE CSV ===
df = pd.read_csv('modified_shopee_reviews.csv')

# === TÊN CỘT NỘI DUNG ĐÁNH GIÁ ===
text_column = 'text'  # ← ĐÃ ĐÚNG: cột tên là 'text'

# Kiểm tra cột tồn tại
if text_column not in df.columns:
    raise KeyError(f"Cột '{text_column}' không tồn tại. Các cột có: {df.columns.tolist()}")

# === HÀM TIỀN XỬ LÝ VĂN BẢN (TIẾNG ANH) ===
def preprocess_text(text):
    if pd.isna(text):
        return ""

    # 1. Chuyển thành chữ thường
    text = str(text).lower()

    # 2. Xóa URL và ký tự đặc biệt (chỉ giữ chữ cái và khoảng trắng)
    text = re.sub(r'http[s]?://\S+|www\.\S+', '', text)  # Xóa URL
    text = re.sub(r'[^\w\s]', '', text)                  # Xóa icon, dấu câu, số

    # 3. Tokenize (tách từ)
    tokens = word_tokenize(text)

    # 4. Loại bỏ stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words and len(word) > 1]

    # 5. Lemmatization (đưa về dạng gốc)
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Ghép lại thành chuỗi
    return ' '.join(tokens)

# === ÁP DỤNG CHO CỘT 'text' → TẠO CỘT MỚI 'clean_review' ===
print(f"Đang xử lý cột: '{text_column}'...")
df['clean_review'] = df[text_column].apply(preprocess_text)

# === LƯU FILE MỚI ===
output_file = 'preprocessed_shopee_reviews.csv'
df.to_csv(output_file, index=False)

# === HOÀN TẤT ===
print("\nHOÀN TẤT TIỀN XỬ LÝ!")
print(f"File đã lưu: {output_file}")
print(f"Tổng số dòng: {len(df):,}")
print(f"Số dòng Pos: {(df['label'] == 'Pos').sum():,}")
print("\nVí dụ 3 dòng đầu (clean_review):")
print(df[['text', 'clean_review']].head(3).to_string(index=False))

Đang xử lý cột: 'text'...

HOÀN TẤT TIỀN XỬ LÝ!
File đã lưu: preprocessed_shopee_reviews.csv
Tổng số dòng: 151,514
Số dòng Pos: 55,000

Ví dụ 3 dòng đầu (clean_review):
                                                                                                                                                                                                            text                                                                                                                                          clean_review
                                                                                                                                                                                                    Awesome ❤️❤️                                                                                                                                               awesome
                                                                                          My daughter love it. Fa

In [23]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# ------------------------------------------------------------------
# 1. Load dữ liệu
# ------------------------------------------------------------------
df = pd.read_csv('preprocessed_shopee_reviews.csv')

# Kiểm tra cột cần thiết
assert 'clean_review' in df.columns, "Cột 'clean_review' không tồn tại!"
assert 'label' in df.columns, "Cột 'label' không tồn tại!"

# Loại bỏ các row không hợp lệ (ví dụ: header bị lẫn hoặc giá trị không phải nhãn thực)
# Giả sử nhãn hợp lệ chỉ là 'Pos', 'Neg', 'Neu'
valid_labels = ['Pos', 'Neg', 'Neu']
df = df[df['label'].isin(valid_labels)]

# Hoặc cụ thể loại bỏ nếu 'label' == 'label'
# df = df[df['label'] != 'label']

X = df['clean_review'].astype(str)   # TF-IDF chấp nhận chuỗi
y = df['label']

print(f"Tổng số mẫu sau lọc: {len(df)}")
print("Phân bố nhãn sau lọc:")
print(y.value_counts())

# ------------------------------------------------------------------
# 2. Xử lý lớp hiếm (có < 2 mẫu) → gộp vào lớp 'Other' nếu cần
# ------------------------------------------------------------------
min_samples = 2
label_counts = Counter(y)

rare_labels = [lbl for lbl, cnt in label_counts.items() if cnt < min_samples]
if rare_labels:
    print(f"\nCác nhãn hiếm (< {min_samples} mẫu): {rare_labels}")
    # Thay thế các nhãn hiếm bằng 'Other'
    y = y.replace(to_replace=rare_labels, value='Other')
    print("Đã gộp các nhãn hiếm thành 'Other'.")
else:
    print("\nKhông có nhãn hiếm nào cần xử lý.")

print("\nPhân bố nhãn cuối cùng:")
print(pd.Series(y).value_counts())

# ------------------------------------------------------------------
# 3. Chia train / test
# ------------------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y          # giữ tỷ lệ nhãn
)

print(f"\nTrain size : {X_train.shape[0]}")
print(f"Test  size : {X_test.shape[0]}")

# ------------------------------------------------------------------
# 4. Vector hóa TF-IDF
# ------------------------------------------------------------------
vectorizer = TfidfVectorizer(
    max_features=10_000,   # giới hạn để chạy nhanh, có thể bỏ nếu RAM đủ
    sublinear_tf=True,     # thường cho hiệu suất tốt hơn
    ngram_range=(1, 1)     # unigram; có thể thử (1,2) nếu muốn
)

# Chỉ fit trên tập train
X_train_tfidf = vectorizer.fit_transform(X_train)

# Transform tập test (không fit lại)
X_test_tfidf = vectorizer.transform(X_test)

print(f"\nX_train_tfidf shape: {X_train_tfidf.shape}")
print(f"X_test_tfidf  shape: {X_test_tfidf.shape}")

# ------------------------------------------------------------------
# 5. (Tuỳ chọn) Lưu vectorizer & dữ liệu đã chia để dùng sau
# ------------------------------------------------------------------
import joblib
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')
joblib.dump((X_train_tfidf, X_test_tfidf, y_train, y_test),
            'tfidf_split_data.pkl')

print("\nĐÃ HOÀN TẤT! Các biến sẵn sàng:")
print("   X_train_tfidf, X_test_tfidf, y_train, y_test")
print("   Vectorizer được lưu: tfidf_vectorizer.pkl")

Tổng số mẫu sau lọc: 151513
Phân bố nhãn sau lọc:
label
Pos    55000
Neu    49083
Neg    47430
Name: count, dtype: int64

Không có nhãn hiếm nào cần xử lý.

Phân bố nhãn cuối cùng:
label
Pos    55000
Neu    49083
Neg    47430
Name: count, dtype: int64

Train size : 121210
Test  size : 30303

X_train_tfidf shape: (121210, 10000)
X_test_tfidf  shape: (30303, 10000)

ĐÃ HOÀN TẤT! Các biến sẵn sàng:
   X_train_tfidf, X_test_tfidf, y_train, y_test
   Vectorizer được lưu: tfidf_vectorizer.pkl


In [24]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

# Assuming the previous code has been run and variables are available: X_train_tfidf, y_train

# Create 3 models
model_nb = MultinomialNB()
model_logreg = LogisticRegression(max_iter=1000)  # Add max_iter to ensure convergence
model_svm = LinearSVC()

print("Đang huấn luyện Naive Bayes...")
model_nb.fit(X_train_tfidf, y_train)
print("Xong!")

print("Đang huấn luyện Logistic Regression...")
model_logreg.fit(X_train_tfidf, y_train)
print("Xong!")

print("Đang huấn luyện SVM (LinearSVC)...")
model_svm.fit(X_train_tfidf, y_train)
print("Xong!")

# Optionally, save the models for later use
import joblib
joblib.dump(model_nb, 'model_nb.pkl')
joblib.dump(model_logreg, 'model_logreg.pkl')
joblib.dump(model_svm, 'model_svm.pkl')

print("All models trained and saved!")

Đang huấn luyện Naive Bayes...
Xong!
Đang huấn luyện Logistic Regression...
Xong!
Đang huấn luyện SVM (LinearSVC)...
Xong!
All models trained and saved!


In [25]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Assuming the previous code has been run and variables are available: model_nb, model_logreg, model_svm, X_test_tfidf, y_test

# Step 1: Make predictions
y_pred_nb = model_nb.predict(X_test_tfidf)
y_pred_logreg = model_logreg.predict(X_test_tfidf)
y_pred_svm = model_svm.predict(X_test_tfidf)

# Step 2: Evaluate each model

# --- Evaluate Naive Bayes ---
print("--- Kết quả của Naive Bayes ---")
print("Accuracy:", accuracy_score(y_test, y_pred_nb))
print(classification_report(y_test, y_pred_nb))

# --- Evaluate Logistic Regression ---
print("\n--- Kết quả của Logistic Regression ---")
print("Accuracy:", accuracy_score(y_test, y_pred_logreg))
print(classification_report(y_test, y_pred_logreg))

# --- Evaluate SVM (LinearSVC) ---
print("\n--- Kết quả của SVM (LinearSVC) ---")
print("Accuracy:", accuracy_score(y_test, y_pred_svm))
print(classification_report(y_test, y_pred_svm))

--- Kết quả của Naive Bayes ---
Accuracy: 0.6690426690426691
              precision    recall  f1-score   support

         Neg       0.68      0.60      0.64      9486
         Neu       0.55      0.59      0.57      9817
         Pos       0.78      0.80      0.79     11000

    accuracy                           0.67     30303
   macro avg       0.67      0.66      0.66     30303
weighted avg       0.67      0.67      0.67     30303


--- Kết quả của Logistic Regression ---
Accuracy: 0.6872256872256872
              precision    recall  f1-score   support

         Neg       0.66      0.68      0.67      9486
         Neu       0.58      0.56      0.57      9817
         Pos       0.80      0.81      0.81     11000

    accuracy                           0.69     30303
   macro avg       0.68      0.68      0.68     30303
weighted avg       0.69      0.69      0.69     30303


--- Kết quả của SVM (LinearSVC) ---
Accuracy: 0.6735306735306735
              precision    recall  f1-sco